Libraries and Style Files
-------------------------

In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

!pip install periodictable
!pip install pyteomics

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

import numpy as np
import pandas as pd
import scipy.constants as co
import ENDF6el as endfel
import masses as ms
import flux_recoil_spec as frs

Objective
-
This notebook builds upon the preceeding notebooks, namely `A-Differential-Neutron-Nucleus-XN.ipynb` and `B-Neutron-Nucleus-Recoil-Distributions.ipynb`. Our objective now is to obtain the neutron flux as a function of energy, reconcile that flux with the differential recoil energy cross section at each energy, and produce a realistic recoil energy distribution as it would appear in a detector.


Load Neutron Flux Data
-

Calculate Differential Recoil Cross Section $\frac{d\sigma}{dE_r}(E_i,E_r)$
-
This section will allow us to determine the liklihood of a neutron with energy $E_i$ scattering off a nucleus and deposit a certain amount of recoil energy E_r. This likelihood is given by the differential cross section $\frac{d\sigma}{dE_r}(E_i,E_r)$. In the previous notebook, `B-Neutron-Nucleus-Recoil-Distributions.ipynb`, we developed code that calculates this value for specific energies. Now, we want to generalize it:

In [2]:
import pandas as pd

fast_flux_df = pd.read_pickle('../data_files/FDF.txt')
print(fast_flux_df.head())

          E      spec
0  0.010000  0.000014
1  0.011665  0.000012
2  0.013330  0.000012
3  0.014995  0.000011
4  0.016660  0.000010
